In [1]:
# import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools
using Base

In [2]:
A = Base.zeros(Float64,50000)
@benchmark A = Array{Float64}(undef, 50000)

BenchmarkTools.Trial: 
  memory estimate:  390.70 KiB
  allocs estimate:  2
  --------------
  minimum time:     977.531 ns (86.62% GC)
  median time:      1.175 μs (87.80% GC)
  mean time:        4.491 μs (96.67% GC)
  maximum time:     142.612 μs (99.85% GC)
  --------------
  samples:          1316
  evals/sample:     852

In [13]:
function f(A)
    for i = 2:length(A)
        A[i] = A[i-1] + 1.2
    end
end

f (generic function with 1 method)

In [14]:
f(A)

In [15]:
A

50000-element Vector{Float64}:
     0.0
     1.2
     2.4
     3.5999999999999996
     4.8
     6.0
     7.2
     8.4
     9.6
    10.799999999999999
    11.999999999999998
    13.199999999999998
    14.399999999999997
     ⋮
 59985.59999994801
 59986.79999994801
 59987.999999948006
 59989.199999948
 59990.399999948
 59991.599999948
 59992.799999947994
 59993.99999994799
 59995.19999994799
 59996.399999947986
 59997.59999994798
 59998.79999994798

In [16]:
@benchmark f(A)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     102.578 μs (0.00% GC)
  median time:      102.650 μs (0.00% GC)
  mean time:        108.327 μs (0.00% GC)
  maximum time:     307.062 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1